# Configure chain internals at runtime

在运行时配置链的内部，通常有两种做法：
- `configurable_fields` 方法：用于配置 `Runnable` 的某个特定域
- `configurable_alternatives` 方法：用于列出 `Runnable` 在运行期可配置的列表

In [8]:
import os

# 初始化
API_KEY = os.getenv("UNION_API_KEY")
BASE_URL = os.getenv("UNION_BASE_URL")

In [1]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [2]:
# 一般调用
model.invoke("Pick a random number")

AIMessage(content='7', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a741bf76-edc4-493b-8c66-633266a8b21d-0')

In [3]:
# 使用config调用
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='91', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b08e149d-4771-4f88-99a7-d7c2b0a675b4-0')

In [4]:
# 一般链式调用

prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

chain.invoke({"x": 0})

AIMessage(content='57', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 14, 'total_tokens': 15}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-09569e5e-8228-4676-b687-ec784e1b74c9-0')

In [5]:
# 使用 config 调用
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 0})

AIMessage(content='94', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 14, 'total_tokens': 15}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c799c0f3-8163-44d2-bbd2-d9bbf3c92936-0')

## 使用 HubRunnables

对于切换提示词很有用！！！

In [6]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

prompt.invoke({"question": "foo", "context": "bar"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:")])

In [7]:
# 切换提示词，commit 对应着 hub 上一个提示词
prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke(
    {"question": "foo", "context": "bar"}
)

ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]")])

## Configurable Alternatives

### 配置大模型的可选项

In [9]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, api_key=API_KEY, base_url=BASE_URL, model='claude-3-haiku-20240307').configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

In [10]:
# By default it will call Anthropic
chain.invoke({"topic": "bears"})

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nHow's that? I tried to come up with a silly pun involving bears. Hopefully you found it at least mildly amusing. Let me know if you'd like to hear another bear-themed joke.", response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 13, 'total_tokens': 86}, 'model_name': 'claude-3-haiku-20240307', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c00ca150-3738-4a41-8990-af57c20251c9-0')

In [11]:
# We can use `.with_config(configurable={"llm": "openai"})` to specify an llm to use
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content='Why did the bear bring a ladder to the bar?\n\nBecause he heard the drinks were on the house!', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-815a2451-4c1d-4a42-9320-a5ee575a1668-0')

In [12]:
# If we use the `default_key` then it uses the default
chain.with_config(configurable={"llm": "anthropic"}).invoke({"topic": "bears"})

AIMessage(content="Here's a bear-themed joke for you:\n\nWhy don't bears wear socks? \nThey have bear feet!", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42, 'total_cost': 3.95e-05}, 'model_name': 'claude-3-haiku-20240307', 'system_fingerprint': None, 'finish_reason': 'end_turn', 'logprobs': None}, id='run-164336ad-bea9-4479-b2b9-f2f77a2ea4df-0')

In [13]:
# If we use the `default_key` then it uses the default
chain.with_config(configurable={"llm": "gpt4"}).invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear socks?\n\nBecause they have bear feet!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bb73138e-822e-4e25-aa40-6c29f725bee9-0')

### 配置 Prompt 的可选项

In [14]:
llm = ChatOpenAI(temperature=0, api_key=API_KEY, base_url=BASE_URL, model='claude-3-haiku-20240307')
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [15]:
# By default it will write a joke
chain.invoke({"topic": "bears"})

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nThey'd get bear feet!\n\nHow's that? I tried to come up with a silly pun-based joke about bears. Puns can be pretty groan-worthy, but that's part of the charm of bear jokes. Let me know if you'd like to hear another one!", response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 13, 'total_tokens': 96, 'total_cost': 0.000107}, 'model_name': 'claude-3-haiku-20240307', 'system_fingerprint': None, 'finish_reason': 'end_turn', 'logprobs': None}, id='run-637e2ab4-0836-4277-a99f-38e3c7a82aee-0')

In [16]:
# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

AIMessage(content="Here is a short poem about bears:\n\nMighty bears, fierce and strong,\nRoaming through the forests long.\nPowerful paws and sharp claws,\nCommanding respect with nature's laws.\n\nGrizzly bears, a sight to behold,\nMajestic and rugged, brave and bold.\nOmnivores thriving in the wild,\nBears, magnificent and untamed and mild.\n\nBeneath the trees, under the moon,\nBears wander, a captivating tune.\nSymbols of nature's raw, untamed grace,\nBears, the lords of their sacred place.", response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 13, 'total_tokens': 151, 'total_cost': 0.00017575}, 'model_name': 'claude-3-haiku-20240307', 'system_fingerprint': None, 'finish_reason': 'end_turn', 'logprobs': None}, id='run-cc7547d3-5a13-4176-a4bb-0bbe40158e9f-0')

### 同时配置`LLM`和`Prompt`选项

In [17]:
claude = ChatOpenAI(temperature=0, api_key=API_KEY, base_url=BASE_URL, model='claude-3-haiku-20240307')

llm = claude.configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)

prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)

chain = prompt | llm

In [18]:
# We can configure it write a poem with OpenAI
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"topic": "bears"}
)

AIMessage(content="In the woods, where sunlight gleams,\nA creature roams, majestic it seems.\nWith fur so thick, and paws so strong,\nThe bear, a symbol of power and song.\n\nThrough valleys deep and mountains tall,\nIt wanders freely, standing proud and tall.\nIts eyes, like amber, reflect the wild,\nA guardian of nature, untamed, undefiled.\n\nWith gentle grace, it fishes the streams,\nIn harmony with nature, fulfilling its dreams.\nA symbol of resilience, it hibernates,\nA testament to strength, it awaits.\n\nYet beneath the might, a gentle soul,\nA mother's love, her cubs she'll hold.\nIn nurturing embrace, they learn and grow,\nGuided by her wisdom, they'll always know.\n\nOh, bear of the wilderness, so wise and true,\nYour presence reminds us of nature's value.\nIn your existence, we find inspiration,\nTo protect this world, with dedication.", response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 13, 'total_tokens': 204}, 'model_name': 'gpt-3.5-turbo', 's

### 保存配置

In [19]:
openai_joke = chain.with_config(configurable={"llm": "openai"})

In [20]:
openai_joke.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ffc8066-edaf-446f-8bab-87e0027bcaec-0')